In [19]:
# Step 1: Import packages
import numpy as np
import scipy
import sympy as sp
import torch
import lightgbm as lgb
import optuna
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_pinball_loss
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import OneHotEncoder
from scipy.stats import friedmanchisquare, wilcoxon
import statsmodels.api as sm
from statsmodels.regression.quantile_regression import QuantReg
import pmlb
from pmlb import fetch_data, regression_dataset_names
import random

# Step 2: Import pysr AFTER Julia dependencies are configured
from pysr import PySRRegressor


In [20]:
def get_feature_type(data_column):
  """
  This function determines the feature type (categorical or binary) for a NumPy array column.

  **Note:** This function assumes the data doesn't contain missing values.
          If your data might have missing values, you'll need to handle them
          before using this function (e.g., impute missing values or remove rows).

  Args:
      data_column: A NumPy array representing the data column.

  Returns:
      A string indicating the feature type: "categorical" or "binary".
  """

  # Check for distinct values and data type
  unique_values = np.unique(data_column)
  num_unique_values = len(unique_values)

  # Categorical data has a limited number of distinct values (adjust threshold as needed)
  if num_unique_values <= 10:  # Adjust threshold based on your data and analysis goals
    return "categorical"

  # Binary data has only two distinct values
  if num_unique_values == 2:
    return "binary"

  # If there are more than 10 distinct values and not binary, assume numerical
  # (This might need further refinement depending on your domain knowledge)
  return "numerical"  # Consider a different label for non-categoric

def get_categorical_features(X):
  """
  This function identifies the indices of categorical features in a NumPy array representing a dataset.

  **Note:** This function assumes the data doesn't contain missing values.
          If your data might have missing values, you'll need to handle them
          before using this function (e.g., impute missing values or remove rows).

  Args:
      X: A 2D NumPy array representing the dataset (n_samples x n_features).

  Returns:
      A list of integers representing the indices of categorical features in X.
  """

  categorical_features = []
  for i, col in enumerate(X.T):  # Enumerate to get column index (i)
    unique_values = np.unique(col)
    num_unique_values = len(unique_values)
    data_type = col.dtype

    # Categorical data has a limited number of distinct values (adjust threshold as needed)
    if num_unique_values <= 10:  # Adjust threshold based on your data and analysis goals
      categorical_features.append(i)

  return categorical_features


def create_dummy_variables(X, categorical_features):
    """
    This function creates dummy variables for categorical features in a dataset.

    Args:
        X: A 2D NumPy array representing the dataset (n_samples x n_features).
        categorical_features: A list of integers representing the indices of categorical features in X.

    Returns:
        A 2D NumPy array representing the dataset with dummy variables for categorical features.
    """
    if categorical_features == []:
        return X
    else:
        # Select categorical features from the data
        X_categorical = X[:, categorical_features]

        # Create one-hot encoder
        encoder = OneHotEncoder(sparse_output=False)

        # Fit the encoder on the categorical features
        encoder.fit(X_categorical)

        # Transform the categorical features into dummy variables
        X_dummy_categorical = encoder.transform(X_categorical)

        # Get the original non-categorical features (assuming they are numerical)
        X_numerical = np.delete(X, categorical_features, axis=1)  # Delete categorical feature columns

        # Combine the dummy variables and numerical features
        X_with_dummies = np.concatenate([X_numerical, X_dummy_categorical], axis=1)

        return X_with_dummies


In [21]:
regression_dataset_namestry = regression_dataset_names
print(regression_dataset_namestry)

['1027_ESL', '1028_SWD', '1029_LEV', '1030_ERA', '1089_USCrime', '1096_FacultySalaries', '1191_BNG_pbc', '1193_BNG_lowbwt', '1196_BNG_pharynx', '1199_BNG_echoMonths', '1201_BNG_breastTumor', '1203_BNG_pwLinear', '1595_poker', '192_vineyard', '195_auto_price', '197_cpu_act', '201_pol', '207_autoPrice', '210_cloud', '215_2dplanes', '218_house_8L', '225_puma8NH', '227_cpu_small', '228_elusage', '229_pwLinear', '230_machine_cpu', '294_satellite_image', '344_mv', '4544_GeographicalOriginalofMusic', '485_analcatdata_vehicle', '503_wind', '505_tecator', '519_vinnie', '522_pm10', '523_analcatdata_neavote', '527_analcatdata_election2000', '529_pollen', '537_houses', '542_pollution', '547_no2', '556_analcatdata_apnea2', '557_analcatdata_apnea1', '560_bodyfat', '561_cpu', '562_cpu_small', '564_fried', '573_cpu_act', '574_house_16H', '579_fri_c0_250_5', '581_fri_c3_500_25', '582_fri_c1_500_25', '583_fri_c1_1000_50', '584_fri_c4_500_25', '586_fri_c3_1000_25', '588_fri_c4_1000_100', '589_fri_c2_1000

SQR BENCHMARK **90TH** QUANTILE

In [ ]:
# Set seed for reproducibility
SEED = 42  # Change as needed

# Set NumPy seed
np.random.seed(SEED)

# Set Python random seed
random.seed(SEED)

# Set PyTorch seed (if using)
torch.manual_seed(SEED)

# Set Optuna seed
optuna.logging.set_verbosity(optuna.logging.WARNING)  # Reduce logging clutter
optuna_seed = SEED

# Global quantile setting (EXCEPT FOR PYSR, THIS NEEDS MANUAL ADJUSTMENT)
QUANTILE = 0.9 #(CHANGE PYSR QUANTILE MANUALLY)

# Function to calculate pinball loss
def pinball_loss(y_true, y_pred, tau=QUANTILE):
    residuals = y_true - y_pred
    loss = np.where(residuals >= 0, tau * residuals, (1 - tau) * -residuals)
    return np.mean(loss)

# Function to calculate normalized pinball loss using the global dataset range
def normalized_pinball_loss(y_true, y_pred, global_min, global_max, tau=QUANTILE):
    range_y = global_max - global_min
    loss = pinball_loss(y_true, y_pred, tau)
    return loss / range_y if range_y != 0 else 0  # Avoid division by zero

# Function to calculate absolute coverage error
def absolute_coverage_error(y_true, y_pred, tau=QUANTILE):
    coverage = np.mean(y_pred >= y_true)
    return np.abs(coverage - tau)

# Function to calculate expression complexity
def calculate_expression_complexity(expression, complexity_of_operators):
    try:
        expr = sp.sympify(expression)
    except sp.SympifyError:
        raise ValueError("Invalid expression")

    complexity = 0
    for atom in sp.preorder_traversal(expr):
        if isinstance(atom, sp.Symbol):  # Variables (e.g., x1, x2)
            complexity += 1
        elif isinstance(atom, (int, float, sp.Integer, sp.Float)):  # Constants
            complexity += 1
        elif atom in complexity_of_operators:  # Operators
            complexity += complexity_of_operators[atom]
    return complexity

# LightGBM objective function for Optuna
def objective_lgb(trial, train_X, train_y, val_X, val_y):
    params = {
        'objective': 'quantile',
        'alpha': QUANTILE,
        'num_leaves': trial.suggest_int('num_leaves', 2, 100),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5, log=True),
        'max_depth': trial.suggest_int('max_depth', 1, 20),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'random_state': SEED,
        'bagging_seed': SEED,
        'feature_fraction_seed': SEED,
        'data_random_seed': SEED,
    }

    if params['min_child_samples'] >= params['num_leaves']:
        raise optuna.exceptions.TrialPruned()

    model = lgb.LGBMRegressor(**params)
    model.fit(train_X, train_y)
    y_pred = model.predict(val_X)
    return pinball_loss(val_y, y_pred, tau=QUANTILE)

# Quantile regression objective function for Optuna
def objective_linear(trial, train_X, train_y, val_X, val_y, tau=QUANTILE):
    max_iter = trial.suggest_int('max_iter', 1000, 5000)
    model = QuantReg(train_y, train_X)
    results = model.fit(q=tau, max_iter=max_iter)
    y_pred = results.predict(val_X)
    return pinball_loss(val_y, y_pred, tau)

# Quantile Decision Tree Regressor
class QuantileDecisionTreeRegressor:
    def __init__(self, quantile=QUANTILE, min_samples_leaf=5, random_state=SEED):
        self.quantile = quantile
        self.min_samples_leaf = min_samples_leaf
        self.tree = DecisionTreeRegressor(min_samples_leaf=min_samples_leaf, random_state=random_state)

    def fit(self, X, y):
        self.tree.fit(X, y)
        self._add_quantile_info(X, y)

    def _add_quantile_info(self, X, y):
        leaf_indices = self.tree.apply(X)
        unique_leaves = np.unique(leaf_indices)
        self.quantile_values = {}
        for leaf in unique_leaves:
            leaf_y = y[leaf_indices == leaf]
            self.quantile_values[leaf] = np.percentile(leaf_y, self.quantile * 100)

    def predict(self, X):
        leaf_indices = self.tree.apply(X)
        predictions = np.array([self.quantile_values[leaf] for leaf in leaf_indices])
        return predictions

# Optuna Objective Function for Decision Tree
def objective_tree(trial, train_X, train_y, val_X, val_y):
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 50)

    model = QuantileDecisionTreeRegressor(quantile=QUANTILE, min_samples_leaf=min_samples_leaf)
    model.fit(train_X, train_y)
    y_pred = model.predict(val_X)

    return pinball_loss(val_y, y_pred, tau=QUANTILE)

# Complexity parameters
binary_operators = ["+", "*", "/", "-"]
unary_operators = ["exp", "sin", "cos", "log", "square"]
complexity_of_operators = {
    "+": 1,
    "-": 1,
    "*": 1,
    "/": 2,
    "exp": 4,
    "sin": 3,
    "cos": 3,
    "log": 3,
    "square": 2,
}

# results90 storage
results90 = {
    "SQR": {"losses": [], "coverage": [], "complexity": []},
    "LightGBM": {"losses": [], "coverage": []},
    "DecisionTree": {"losses": [], "coverage": [], "complexity": []},
    "LinearQuantile": {"losses": [], "coverage": [], "complexity": []},
}

def process_fold_scores(model_name, fold_scores):
    for metric, scores in fold_scores.items():
        results90[model_name][metric].extend(scores)


# Iterate over datasets
for regression_dataset in regression_dataset_namestry:
    try:
        print(regression_dataset)
        X1, y = fetch_data(regression_dataset, return_X_y=True)
        global_min, global_max = np.min(y), np.max(y)  # Global range for determ. normalization

        X = create_dummy_variables(X1, get_categorical_features(X1))

        kf = KFold(n_splits=5, shuffle=True, random_state=SEED)

        fold_scores_sqr = {"losses": [], "coverage": [], "complexity": []}
        fold_scores_lgb = {"losses": [], "coverage": []}
        fold_scores_tree = {"losses": [], "coverage": [], "complexity": []}
        fold_scores_linear = {"losses": [], "coverage": [], "complexity": []}

        for train_index, test_index in kf.split(X):
            train_X, test_X = X[train_index], X[test_index]
            train_y, test_y = y[train_index], y[test_index]

            # Symbolic Quantile Regression
            modelq = PySRRegressor(
                niterations=900, #imrpove for better results90
                binary_operators=binary_operators,
                unary_operators=unary_operators,
                complexity_of_operators=complexity_of_operators,
                elementwise_loss="pinball_loss(y_true, y_pred) = max.(0.1 * (y_true - y_pred), (0.1 - 1) * (y_true - y_pred))", #DONT FORGET TO CHANGE WHEN CHANGING QUANTILE (JULIA SYNTAX)
                temp_equation_file=True,
                random_state=SEED
            )

            modelq.fit(train_X, train_y)
            y_pred_symbolic = modelq.predict(test_X)

            # Metrics for SQR
            fold_scores_sqr["losses"].append(normalized_pinball_loss(test_y, y_pred_symbolic, global_min, global_max))
            fold_scores_sqr["coverage"].append(absolute_coverage_error(test_y, y_pred_symbolic))
            fold_scores_sqr["complexity"].append(calculate_expression_complexity(modelq.sympy(), complexity_of_operators))

            # LightGBM Quantile Regression
            study_lgb = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=SEED))
            study_lgb.optimize(lambda trial: objective_lgb(trial, train_X, train_y, test_X, test_y), n_trials=10)

            best_params_lgb = study_lgb.best_params
            model_lgb = lgb.LGBMRegressor(objective='quantile', alpha=QUANTILE, **best_params_lgb)
            model_lgb.fit(train_X, train_y)
            y_pred_lgb = model_lgb.predict(test_X)

            # Metrics for LightGBM
            fold_scores_lgb["losses"].append(normalized_pinball_loss(test_y, y_pred_lgb, global_min, global_max))
            fold_scores_lgb["coverage"].append(absolute_coverage_error(test_y, y_pred_lgb))

            # Inside the main loop for dataset processing (NEW)
            study_tree = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=SEED))
            study_tree.optimize(lambda trial: objective_tree(trial, train_X, train_y, test_X, test_y), n_trials=10)

            best_params_tree = study_tree.best_params  # Get best hyperparameter

            # Train the best Decision Tree model with optimized min_samples_leaf
            model_tree = QuantileDecisionTreeRegressor(quantile=QUANTILE, min_samples_leaf=best_params_tree['min_samples_leaf'])
            model_tree.fit(train_X, train_y)
            y_pred_tree = model_tree.predict(test_X)

            # Metrics for Decision Tree (NEW complexity calculation)
            fold_scores_tree["losses"].append(normalized_pinball_loss(test_y, y_pred_tree, global_min, global_max))
            fold_scores_tree["coverage"].append(absolute_coverage_error(test_y, y_pred_tree))
            fold_scores_tree["complexity"].append(model_tree.tree.tree_.node_count)  # NEW: Store tree complexity

            # Linear Quantile Regression
            study_linear = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=SEED))
            study_linear.optimize(lambda trial: objective_linear(trial, train_X, train_y, test_X, test_y), n_trials=10)

            best_params_linear = study_linear.best_params
            model_linear = QuantReg(train_y, train_X).fit(q=QUANTILE, max_iter=best_params_linear['max_iter'])
            y_pred_linear = model_linear.predict(test_X)

            # Metrics for Linear Quantile Regression
            fold_scores_linear["losses"].append(normalized_pinball_loss(test_y, y_pred_linear, global_min, global_max))
            fold_scores_linear["coverage"].append(absolute_coverage_error(test_y, y_pred_linear))
            fold_scores_linear["complexity"].append(X.shape[1])

        process_fold_scores("SQR", fold_scores_sqr)
        process_fold_scores("LightGBM", fold_scores_lgb)
        process_fold_scores("DecisionTree", fold_scores_tree)
        process_fold_scores("LinearQuantile", fold_scores_linear)
    except Exception as e:
        print(f"Error processing {regression_dataset}: {e}")

# Display results90
print(results90)


1027_ESL


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pysr/sr.py:2780: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pysr/sr.py:2265: UserWarning: Note: you are running with 10 features or more. Genetic algorithms like used in PySR scale poorly with large numbers of features. You should run PySR for more `niterations` to ensure it can find the correct variables, and consider using a larger `maxsize`.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pysr/sr.py:1844: UserWarning: Note: Setting `random_state` without also setting `deterministic=True` and `parallelism='serial'` will result in non-deterministic searches.
  warnings.warn(


In [ ]:
# Extract all available metrics dynamically
available_metrics = set()
for model in results90:
    available_metrics.update(results90[model].keys())
metrics = list(available_metrics)  # Ensures only existing metrics are used

models = list(results90.keys())  # ["SQR", "LightGBM", "DecisionTree", "LinearQuantile"]

# Significance level for Bonferroni correction
alpha = 0.05

# Store results
friedman_results = {}
wilcoxon_results = {}

# Perform Friedman test for each available metric
for metric in metrics:
    # Filter models that contain this metric
    valid_models = [model for model in models if metric in results90[model]]

    # Gather data for each valid model
    data = [results90[model][metric] for model in valid_models]

    # Perform Friedman test (only if multiple models have the metric)
    if len(data) > 1:
        stat, p_value = friedmanchisquare(*data)
        friedman_results[metric] = {'statistic': stat, 'p_value': p_value}

        # Check for significance
        wilcoxon_results[metric] = {}  # Initialize results for this metric

        # Perform pairwise Wilcoxon tests with Bonferroni correction
        num_comparisons = len(valid_models) * (len(valid_models) - 1) // 2
        corrected_alpha = alpha / num_comparisons if num_comparisons > 0 else alpha

        for i, model_1 in enumerate(valid_models):
            for j, model_2 in enumerate(valid_models):
                if i < j:  # Avoid duplicate comparisons
                    stat, p_value = wilcoxon(results90[model_1][metric], results90[model_2][metric])
                    wilcoxon_results[metric][f"{model_1} vs {model_2}"] = {
                        'statistic': stat,
                        'p_value': p_value,
                        'significant': p_value < corrected_alpha
                    }

# Display results
print("Friedman Test Results:")
for metric, result in friedman_results.items():
    significance = "Significant" if result['p_value'] < alpha else "Not Significant"
    print(f"Metric: {metric}, Statistic: {result['statistic']}, p-value: {result['p_value']} ({significance})")

print("\nWilcoxon Test Results (All Comparisons):")
for metric, comparisons in wilcoxon_results.items():
    print(f"\nMetric: {metric}")
    for comparison, result in comparisons.items():
        significance = "Significant" if result['significant'] else "Not Significant"
        print(f"  {comparison}: Statistic: {result['statistic']}, p-value: {result['p_value']} ({significance})")


SQR BENCHMARK **50TH** QUANTILE

In [ ]:
# Set seed for reproducibility
SEED = 42  # Change as needed

# Set NumPy seed
np.random.seed(SEED)

# Set Python random seed
random.seed(SEED)

# Set PyTorch seed (if using)
torch.manual_seed(SEED)

# Set Optuna seed
optuna.logging.set_verbosity(optuna.logging.WARNING)  # Reduce logging clutter
optuna_seed = SEED

# Global quantile setting (EXCEPT FOR PYSR, THIS NEEDS MANUAL ADJUSTMENT)
QUANTILE = 0.5 #(CHANGE PYSR QUANTILE MANUALLY)

# Function to calculate pinball loss
def pinball_loss(y_true, y_pred, tau=QUANTILE):
    residuals = y_true - y_pred
    loss = np.where(residuals >= 0, tau * residuals, (1 - tau) * -residuals)
    return np.mean(loss)

# Function to calculate normalized pinball loss using the global dataset range
def normalized_pinball_loss(y_true, y_pred, global_min, global_max, tau=QUANTILE):
    range_y = global_max - global_min
    loss = pinball_loss(y_true, y_pred, tau)
    return loss / range_y if range_y != 0 else 0  # Avoid division by zero

# Function to calculate absolute coverage error
def absolute_coverage_error(y_true, y_pred, tau=QUANTILE):
    coverage = np.mean(y_pred >= y_true)
    return np.abs(coverage - tau)

# Function to calculate expression complexity
def calculate_expression_complexity(expression, complexity_of_operators):
    try:
        expr = sp.sympify(expression)
    except sp.SympifyError:
        raise ValueError("Invalid expression")

    complexity = 0
    for atom in sp.preorder_traversal(expr):
        if isinstance(atom, sp.Symbol):  # Variables (e.g., x1, x2)
            complexity += 1
        elif isinstance(atom, (int, float, sp.Integer, sp.Float)):  # Constants
            complexity += 1
        elif atom in complexity_of_operators:  # Operators
            complexity += complexity_of_operators[atom]
    return complexity

# LightGBM objective function for Optuna
def objective_lgb(trial, train_X, train_y, val_X, val_y):
    params = {
        'objective': 'quantile',
        'alpha': QUANTILE,
        'num_leaves': trial.suggest_int('num_leaves', 2, 100),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5, log=True),
        'max_depth': trial.suggest_int('max_depth', 1, 20),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'random_state': SEED,
        'bagging_seed': SEED,
        'feature_fraction_seed': SEED,
        'data_random_seed': SEED,
    }

    if params['min_child_samples'] >= params['num_leaves']:
        raise optuna.exceptions.TrialPruned()

    model = lgb.LGBMRegressor(**params)
    model.fit(train_X, train_y)
    y_pred = model.predict(val_X)
    return pinball_loss(val_y, y_pred, tau=QUANTILE)

# Quantile regression objective function for Optuna
def objective_linear(trial, train_X, train_y, val_X, val_y, tau=QUANTILE):
    max_iter = trial.suggest_int('max_iter', 1000, 5000)
    model = QuantReg(train_y, train_X)
    results50 = model.fit(q=tau, max_iter=max_iter)
    y_pred = results50.predict(val_X)
    return pinball_loss(val_y, y_pred, tau)

# Quantile Decision Tree Regressor
class QuantileDecisionTreeRegressor:
    def __init__(self, quantile=QUANTILE, min_samples_leaf=5, random_state=SEED):
        self.quantile = quantile
        self.min_samples_leaf = min_samples_leaf
        self.tree = DecisionTreeRegressor(min_samples_leaf=min_samples_leaf, random_state=random_state)

    def fit(self, X, y):
        self.tree.fit(X, y)
        self._add_quantile_info(X, y)

    def _add_quantile_info(self, X, y):
        leaf_indices = self.tree.apply(X)
        unique_leaves = np.unique(leaf_indices)
        self.quantile_values = {}
        for leaf in unique_leaves:
            leaf_y = y[leaf_indices == leaf]
            self.quantile_values[leaf] = np.percentile(leaf_y, self.quantile * 100)

    def predict(self, X):
        leaf_indices = self.tree.apply(X)
        predictions = np.array([self.quantile_values[leaf] for leaf in leaf_indices])
        return predictions

# Optuna Objective Function for Decision Tree
def objective_tree(trial, train_X, train_y, val_X, val_y):
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 50)

    model = QuantileDecisionTreeRegressor(quantile=QUANTILE, min_samples_leaf=min_samples_leaf)
    model.fit(train_X, train_y)
    y_pred = model.predict(val_X)

    return pinball_loss(val_y, y_pred, tau=QUANTILE)

# Complexity parameters
binary_operators = ["+", "*", "/", "-"]
unary_operators = ["exp", "sin", "cos", "log", "square"]
complexity_of_operators = {
    "+": 1,
    "-": 1,
    "*": 1,
    "/": 2,
    "exp": 4,
    "sin": 3,
    "cos": 3,
    "log": 3,
    "square": 2,
}

# results50 storage
results50 = {
    "SQR": {"losses": [], "coverage": [], "complexity": []},
    "LightGBM": {"losses": [], "coverage": []},
    "DecisionTree": {"losses": [], "coverage": [], "complexity": []},
    "LinearQuantile": {"losses": [], "coverage": [], "complexity": []},
}

def process_fold_scores(model_name, fold_scores):
    for metric, scores in fold_scores.items():
        results50[model_name][metric].extend(scores)


# Iterate over datasets
for regression_dataset in regression_dataset_namestry:
    try:
        print(regression_dataset)
        X1, y = fetch_data(regression_dataset, return_X_y=True)
        global_min, global_max = np.min(y), np.max(y)  # Global range for determ. normalization

        X = create_dummy_variables(X1, get_categorical_features(X1))

        kf = KFold(n_splits=5, shuffle=True, random_state=SEED)

        fold_scores_sqr = {"losses": [], "coverage": [], "complexity": []}
        fold_scores_lgb = {"losses": [], "coverage": []}
        fold_scores_tree = {"losses": [], "coverage": [], "complexity": []}
        fold_scores_linear = {"losses": [], "coverage": [], "complexity": []}

        for train_index, test_index in kf.split(X):
            train_X, test_X = X[train_index], X[test_index]
            train_y, test_y = y[train_index], y[test_index]

            # Symbolic Quantile Regression
            modelq = PySRRegressor(
                niterations=900, #imrpove for better results50
                binary_operators=binary_operators,
                unary_operators=unary_operators,
                complexity_of_operators=complexity_of_operators,
                elementwise_loss="pinball_loss(y_true, y_pred) = max.(0.5 * (y_true - y_pred), (0.5 - 1) * (y_true - y_pred))", #DONT FORGET TO CHANGE WHEN CHANGING QUANTILE (JULIA SYNTAX)
                temp_equation_file=True,
                random_state=SEED
            )

            modelq.fit(train_X, train_y)
            y_pred_symbolic = modelq.predict(test_X)

            # Metrics for SQR
            fold_scores_sqr["losses"].append(normalized_pinball_loss(test_y, y_pred_symbolic, global_min, global_max))
            fold_scores_sqr["coverage"].append(absolute_coverage_error(test_y, y_pred_symbolic))
            fold_scores_sqr["complexity"].append(calculate_expression_complexity(modelq.sympy(), complexity_of_operators))

            # LightGBM Quantile Regression
            study_lgb = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=SEED))
            study_lgb.optimize(lambda trial: objective_lgb(trial, train_X, train_y, test_X, test_y), n_trials=10)

            best_params_lgb = study_lgb.best_params
            model_lgb = lgb.LGBMRegressor(objective='quantile', alpha=QUANTILE, **best_params_lgb)
            model_lgb.fit(train_X, train_y)
            y_pred_lgb = model_lgb.predict(test_X)

            # Metrics for LightGBM
            fold_scores_lgb["losses"].append(normalized_pinball_loss(test_y, y_pred_lgb, global_min, global_max))
            fold_scores_lgb["coverage"].append(absolute_coverage_error(test_y, y_pred_lgb))

            # Inside the main loop for dataset processing (NEW)
            study_tree = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=SEED))
            study_tree.optimize(lambda trial: objective_tree(trial, train_X, train_y, test_X, test_y), n_trials=10)

            best_params_tree = study_tree.best_params  # Get best hyperparameter

            # Train the best Decision Tree model with optimized min_samples_leaf
            model_tree = QuantileDecisionTreeRegressor(quantile=QUANTILE, min_samples_leaf=best_params_tree['min_samples_leaf'])
            model_tree.fit(train_X, train_y)
            y_pred_tree = model_tree.predict(test_X)

            # Metrics for Decision Tree (NEW complexity calculation)
            fold_scores_tree["losses"].append(normalized_pinball_loss(test_y, y_pred_tree, global_min, global_max))
            fold_scores_tree["coverage"].append(absolute_coverage_error(test_y, y_pred_tree))
            fold_scores_tree["complexity"].append(model_tree.tree.tree_.node_count)  # NEW: Store tree complexity

            # Linear Quantile Regression
            study_linear = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=SEED))
            study_linear.optimize(lambda trial: objective_linear(trial, train_X, train_y, test_X, test_y), n_trials=10)

            best_params_linear = study_linear.best_params
            model_linear = QuantReg(train_y, train_X).fit(q=QUANTILE, max_iter=best_params_linear['max_iter'])
            y_pred_linear = model_linear.predict(test_X)

            # Metrics for Linear Quantile Regression
            fold_scores_linear["losses"].append(normalized_pinball_loss(test_y, y_pred_linear, global_min, global_max))
            fold_scores_linear["coverage"].append(absolute_coverage_error(test_y, y_pred_linear))
            fold_scores_linear["complexity"].append(X.shape[1])

        process_fold_scores("SQR", fold_scores_sqr)
        process_fold_scores("LightGBM", fold_scores_lgb)
        process_fold_scores("DecisionTree", fold_scores_tree)
        process_fold_scores("LinearQuantile", fold_scores_linear)
    except Exception as e:
        print(f"Error processing {regression_dataset}: {e}")

# Display results50
print(results50)


1029_LEV


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pysr/sr.py:2780: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pysr/sr.py:2265: UserWarning: Note: you are running with 10 features or more. Genetic algorithms like used in PySR scale poorly with large numbers of features. You should run PySR for more `niterations` to ensure it can find the correct variables, and consider using a larger `maxsize`.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pysr/sr.py:1844: UserWarning: Note: Setting `random_state` without also setting `deterministic=True` and `parallelism='serial'` will result in non-deterministic searches.
  warnings.warn(
[ Info: Started!



Expressions evaluated per second: 1.910e+05
Progress: 978 / 3100 total iterations (31.548%)
════════════════════════════════════════════════════════════════════════════════════════════════════
───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           3.656e-01  1.594e+01  y = 2
3           3.169e-01  7.155e-02  y = 2 - x₅
5           2.894e-01  4.539e-02  y = 2 - (x₅ + x₆)
7           2.713e-01  3.234e-02  y = (2 - (x₅ + x₀)) + x₉
9           2.613e-01  1.878e-02  y = x₄ + (x₃ + ((2 - x₅) - x₆))
11          2.500e-01  2.201e-02  y = (x₄ + (((x₃ - x₅) - x₁₅) + 2)) - x₆
14          2.491e-01  1.151e-03  y = (cos(x₆) * (((2 - x₅) - x₀) + x₉)) + x₄
16          2.465e-01  5.290e-03  y = (cos(x₆) * ((((2 - x₅) - x₁₅) + x₉) + x₃)) + x₄
19          2.464e-01  1.926e-04  y = (cos(x₆) * (((x₃ + x₉) + (2 - x₅)) - x₁₅)) + (x₄ / 1.0...
                                      82)
21          2.449e-01  2.

[ Info: Final population:
[ Info: Results saved to:
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           3.656e-01  1.594e+01  y = 2
3           3.169e-01  7.155e-02  y = 2 - x₅
5           2.894e-01  4.539e-02  y = 2 - (x₅ + x₆)
7           2.713e-01  3.234e-02  y = (2 - (x₅ + x₀)) + x₉
9           2.581e-01  2.480e-02  y = (x₉ + (x₈ + (2 - x₅))) - x₀
11          2.500e-01  1.599e-02  y = (x₄ + (((x₃ - x₅) - x₁₅) + 2)) - x₆
13          2.438e-01  1.266e-02  y = ((x₄ + (((x₃ - x₅) - x₁₅) + 2)) - x₆) + x₉
17          2.407e-01  3.140e-03  y = (((x₄ + ((x₁₅ * x₃) + 1.9998)) + (x₆ * -1.0003)) - x₅)...
                                       + (x₉ - x₁₅)
18          2.407e-01  1.575e-04  y = ((x₄ + ((x₁₅ * x₃) + x₉)) - x₁₅) + ((x₆ / -0.99946) + ...
                                      (2 - x₅))
19          2.357e-01  2.075e-02  y = (x₁₅ * (x₃ + x₁₂)) + (((x₉ - x₁₅) + (x₄ + ((-1.0015 * ...
                                      x₆) + 2))) - x

...

  - /var/folders/v0/2pg4g4d55fjcq1qrmyjmjp680000gp/T/tmpihqjezx2/20250304_163115_Ik9A91/hall_of_fame.csv

Expressions evaluated per second: 2.290e+05
Progress: 1254 / 3100 total iterations (40.452%)
════════════════════════════════════════════════════════════════════════════════════════════════════
───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           3.525e-01  1.594e+01  y = 2
3           3.131e-01  5.922e-02  y = 2 - x₅
5           2.919e-01  3.514e-02  y = (2 - x₅) - x₆
7           2.750e-01  2.978e-02  y = ((2 - x₅) - x₆) + x₄
9           2.738e-01  2.277e-03  y = ((x₉ - x₀) + 2) - (x₅ - x₈)
11          2.594e-01  2.697e-02  y = ((x₄ + (x₃ + x₈)) + (x₉ + x₇)) + 1
13          2.506e-01  1.711e-02  y = ((((x₄ + x₈) + x₉) + x₇) + (x₃ + 0.99993)) - x₁₅
15          2.413e-01  1.903e-02  y = (x₄ + x₉) + ((x₃ + (0.99985 + (x₈ + x₇))) - (x₀ * x₁₅)...
                                      )

[ Info: Final population:
[ Info: Results saved to:
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000067 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 40
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 20
[LightGBM] [Info] Start training from score 2.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


{'SQR': {'losses': [np.float64(0.07750000687500001), np.float64(0.075625), np.float64(0.0775), np.float64(0.0825), np.float64(0.08125)], 'coverage': [np.float64(0.275), np.float64(0.265), np.float64(0.25), np.float64(0.25), np.float64(0.24)], 'complexity': [3, 3, 3, 3, 3]}, 'LightGBM': {'losses': [np.float64(0.0676338824688943), np.float64(0.0641968982293419), np.float64(0.06480703525507796), np.float64(0.06753785037244704), np.float64(0.06619738926178854)], 'coverage': [np.float64(0.030000000000000027), np.float64(0.04500000000000004), np.float64(0.040000000000000036), np.float64(0.09499999999999997), np.float64(0.050000000000000044)]}, 'DecisionTree': {'losses': [np.float64(0.05875), np.float64(0.0490625), np.float64(0.04375), np.float64(0.0490625), np.float64(0.0484375)], 'coverage': [np.float64(0.235), np.float64(0.32499999999999996), np.float64(0.33999999999999997), np.float64(0.275), np.float64(0.32999999999999996)], 'complexity': [179, 183, 175, 117, 175]}, 'LinearQuantile': {'l

In [ ]:
# Extract all available metrics dynamically
available_metrics = set()
for model in results50:
    available_metrics.update(results50[model].keys())
metrics = list(available_metrics)  # Ensures only existing metrics are used

models = list(results50.keys())  # ["SQR", "LightGBM", "DecisionTree", "LinearQuantile"]

# Significance level for Bonferroni correction
alpha = 0.05

# Store results
friedman_results = {}
wilcoxon_results = {}

# Perform Friedman test for each available metric
for metric in metrics:
    # Filter models that contain this metric
    valid_models = [model for model in models if metric in results50[model]]

    # Gather data for each valid model
    data = [results50[model][metric] for model in valid_models]

    # Perform Friedman test (only if multiple models have the metric)
    if len(data) > 1:
        stat, p_value = friedmanchisquare(*data)
        friedman_results[metric] = {'statistic': stat, 'p_value': p_value}

        # Check for significance
        wilcoxon_results[metric] = {}  # Initialize results for this metric

        # Perform pairwise Wilcoxon tests with Bonferroni correction
        num_comparisons = len(valid_models) * (len(valid_models) - 1) // 2
        corrected_alpha = alpha / num_comparisons if num_comparisons > 0 else alpha

        for i, model_1 in enumerate(valid_models):
            for j, model_2 in enumerate(valid_models):
                if i < j:  # Avoid duplicate comparisons
                    stat, p_value = wilcoxon(results50[model_1][metric], results50[model_2][metric])
                    wilcoxon_results[metric][f"{model_1} vs {model_2}"] = {
                        'statistic': stat,
                        'p_value': p_value,
                        'significant': p_value < corrected_alpha
                    }

# Display results
print("Friedman Test Results:")
for metric, result in friedman_results.items():
    significance = "Significant" if result['p_value'] < alpha else "Not Significant"
    print(f"Metric: {metric}, Statistic: {result['statistic']}, p-value: {result['p_value']} ({significance})")

print("\nWilcoxon Test Results (All Comparisons):")
for metric, comparisons in wilcoxon_results.items():
    print(f"\nMetric: {metric}")
    for comparison, result in comparisons.items():
        significance = "Significant" if result['significant'] else "Not Significant"
        print(f"  {comparison}: Statistic: {result['statistic']}, p-value: {result['p_value']} ({significance})")


In [17]:

# Merge both results into a dictionary with dataset labels
all_results = {"results90": results90, "results50": results50}

# Create a structured dictionary for easy merging
data_list = []
num_entries = len(next(iter(results90.values()))["losses"])  # Assume same number of entries for all models

for dataset, results in all_results.items():
    for i in range(num_entries):  # Iterate over the index of each loss/coverage/complexity entry
        row = {"Dataset": dataset}
        for model, metrics in results.items():
            row[f"{model}Loss"] = float(metrics["losses"][i])
            row[f"{model}Coverage"] = float(metrics["coverage"][i])
            row[f"{model}Complexity"] = metrics.get("complexity", [None] * num_entries)[i]
        data_list.append(row)

# Convert list to DataFrame
df = pd.DataFrame(data_list)

# Save to CSV with correct decimal formatting
df.to_csv("results.csv", index=False, sep=",", decimal=".")

print("CSV file saved successfully.")


CSV file saved successfully.


In [18]:
with open("results.csv", "r", encoding="utf-8") as f:
    for _ in range(10):  # Print first 5 lines
        print(f.readline())


Dataset,SQRLoss,SQRCoverage,SQRComplexity,LightGBMLoss,LightGBMCoverage,LightGBMComplexity,DecisionTreeLoss,DecisionTreeCoverage,DecisionTreeComplexity,LinearQuantileLoss,LinearQuantileCoverage,LinearQuantileComplexity

results90,0.024000045499999997,0.06999999999999995,4,0.027760094816986824,0.015000000000000013,,0.027150000000000004,0.05999999999999994,23,0.02701982968444768,0.04500000000000004,20

results90,0.031125041249999992,0.05499999999999994,4,0.030983229297781804,0.025000000000000022,,0.026775000000000004,0.07499999999999996,57,0.031166536302087787,0.040000000000000036,20

results90,0.028000170999999997,0.05999999999999994,4,0.029743355494453675,0.0050000000000000044,,0.025824999999999997,0.07999999999999996,21,0.031093676142103108,0.025000000000000022,20

results90,0.025631369999999994,0.06999999999999995,5,0.028077458473769978,0.0050000000000000044,,0.0218375,0.06499999999999995,117,0.029265484826350982,0.04500000000000004,20

results90,0.030375335249999996,0.04499999999999